In [1]:
#Import dependencies
import pandas as pd
import sqlite3
import argparse
import json
import pprint
import requests
import sys
import urllib
from urllib.error import HTTPError
from urllib.parse import quote
from urllib.parse import urlencode
import matplotlib.pyplot as plt
import csv
import os

In [9]:
#Assign database file to a variable
db_file = "../Resources/Los_Angeles/crimedata.db"

#Connect to database file
conn = sqlite3.connect(db_file)

In [10]:
#Query the la crime data table and put into a pandas dataframe
crime_df = pd.read_sql_query("select * from la_crime_1718;", conn)
crime_df.head()

,DRNumber,DateReported,DateOccurred,TimeOccurred,AreaID,AreaName,ReportingDistrict,CrimeCode,CrimeCodeDescription,MOCodes,...,WeaponDescription,StatusCode,StatusDescription,CrimeCode1,CrimeCode2,CrimeCode3,CrimeCode4,Address,CrossStreet,Location
0,170209449,4/22/2017,4/21/2017,1930,2,Rampart,201,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,ALEXANDRIA,ROMAINE,"(34.0886, -118.2979)"
1,170206243,2/14/2017,2/11/2017,1700,2,Rampart,275,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,OLYMPIC,WESTLAKE,"(34.0512, -118.2787)"
2,170311599,4/25/2017,4/25/2017,745,3,Southwest,325,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,VERMONT,ADAMS,"(34.0328, -118.2915)"
3,170408264,4/7/2017,4/7/2017,1,4,Hollenbeck,421,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,ALHAMBRA,VALLEY,"(34.0676, -118.2202)"
4,170508584,4/10/2017,4/8/2017,730,5,Harbor,566,510,VEHICLE - STOLEN,,...,,IC,Invest Cont,510,,,,10TH,CENTURY,"(33.7347, -118.2842)"


In [11]:
crime_df.columns

Index(['DRNumber', 'DateReported', 'DateOccurred', 'TimeOccurred', 'AreaID',
       'AreaName', 'ReportingDistrict', 'CrimeCode', 'CrimeCodeDescription',
       'MOCodes', 'VictimAge', 'VictimSex', 'VictimDescent', 'PremiseCode',
       'PremiseDescription', 'WeaponUsedCode', 'WeaponDescription',
       'StatusCode', 'StatusDescription', 'CrimeCode1', 'CrimeCode2',
       'CrimeCode3', 'CrimeCode4', 'Address', 'CrossStreet', 'Location'],
      dtype='object')

In [12]:
# Drop unnecessary crime columns for machine learning
crime_df = crime_df.drop(['DRNumber','DateReported', 
                          'DateOccurred','TimeOccurred',
                          'AreaID','ReportingDistrict','CrimeCode',
                          'MOCodes','VictimSex',
                          'WeaponUsedCode','PremiseCode','PremiseDescription',
                          'WeaponDescription','VictimDescent',
                          'StatusCode', 'StatusDescription',
                          'Address', 'CrossStreet',
                          'CrimeCode1', 'CrimeCode2',
                          'CrimeCode3', 'CrimeCode4','Location'
                         ], axis=1)
crime_df.head()

,AreaName,CrimeCodeDescription,VictimAge
0,Rampart,VEHICLE - STOLEN,16
1,Rampart,VEHICLE - STOLEN,16
2,Southwest,VEHICLE - STOLEN,16
3,Hollenbeck,VEHICLE - STOLEN,16
4,Harbor,VEHICLE - STOLEN,16


In [13]:
crime_df.count()

AreaName                174766
CrimeCodeDescription    174766
VictimAge               174766
dtype: int64

In [14]:
crime_df.dropna(inplace = True) 

In [15]:
crime_df.count()

AreaName                174766
CrimeCodeDescription    174766
VictimAge               174766
dtype: int64

In [16]:
crime_df.to_csv("../Resources/Los_Angeles/la_tableau.csv",index=False)

In [12]:
crime_df.count()

AreaName                174766
CrimeCodeDescription    174766
VictimAge               174766
dtype: int64

In [13]:
# Most victim age crimes are commited against
ages = crime_df.VictimAge.value_counts()
ages.head(10)

17    12516
16    12093
      11503
26     4411
27     4323
25     4271
28     4179
29     4131
24     4063
30     3966
Name: VictimAge, dtype: int64

In [14]:
area_name = crime_df.AreaName.value_counts() 
area_name.head(10)

Southwest     26280
Central       24183
Hollywood     20940
Wilshire      18542
Rampart       18192
Van Nuys      17306
Harbor        16300
Hollenbeck    15293
West LA       15290
Newton          816
Name: AreaName, dtype: int64

In [15]:
crime_code_desc = crime_df.CrimeCodeDescription.value_counts() 
crime_code_desc.head(10)

BATTERY - SIMPLE ASSAULT                           16820
BURGLARY FROM VEHICLE                              14673
THEFT PLAIN - PETTY ($950 & UNDER)                 13046
VEHICLE - STOLEN                                   11412
BURGLARY                                           10286
INTIMATE PARTNER - SIMPLE ASSAULT                   8903
ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT      8644
ROBBERY                                             7344
THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER)     6942
THEFT OF IDENTITY                                   6670
Name: CrimeCodeDescription, dtype: int64

In [23]:
# Use Pandas get_dummies to convert categorical data
data_binary_encoded = pd.get_dummies(crime_df, columns=["AreaName","CrimeCodeDescription"])
data_binary_encoded.head()

,VictimAge,AreaName_77th Street,AreaName_Central,AreaName_Harbor,AreaName_Hollenbeck,AreaName_Hollywood,AreaName_Mission,AreaName_N Hollywood,AreaName_Newton,AreaName_Northeast,...,CrimeCodeDescription_UNAUTHORIZED COMPUTER ACCESS,"CrimeCodeDescription_VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS)","CrimeCodeDescription_VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) 0114",CrimeCodeDescription_VANDALISM - MISDEAMEANOR ($399 OR UNDER),CrimeCodeDescription_VEHICLE - ATTEMPT STOLEN,CrimeCodeDescription_VEHICLE - STOLEN,CrimeCodeDescription_VIOLATION OF COURT ORDER,CrimeCodeDescription_VIOLATION OF RESTRAINING ORDER,CrimeCodeDescription_VIOLATION OF TEMPORARY RESTRAINING ORDER,CrimeCodeDescription_WEAPONS POSSESSION/BOMBING
0,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,16,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,16,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,16,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [25]:
data_binary_encoded.to_csv("../Resources/Los_Angeles/encoded_areaname_crimecode.csv",index=False)